## Combining DataFrames

Previously, we have seen how to analyze and manipulate data in a single DataFrame. However, you will often find data saved into different files and, therefore, you may need to deal with several different pandas DataFrames.<br>
In this session we will explore different ways of combining DataFrames into a single DataFrame.

Let's start loading the `pandas` library, reading two data sets into DataFrames, and having a quick look at the tabular data: ```surveys.csv``` and ```species.csv```

In [ ]:
import pandas as pd

In [ ]:
surveys_df = pd.read_csv("../course_materials/data/surveys.csv", keep_default_na=False, na_values=[""])
species_df = pd.read_csv("../course_materials/data/species.csv", keep_default_na=False, na_values=[""])

In [ ]:
print(surveys_df.info())
surveys_df.head()

In [ ]:
print(species_df.info())
species_df.head()

We now have two DataFrames. The first, `surveys_df`, contains information on individuals of a species recorded in a survey, while the second, `species_df`, contains more detailed information on each species.

## Concatenating DataFrames

The first way we will combine DataFrames is **concatenation**, i.e. simply putting DataFrames one after the other either **verically** or **horizontally**.

Concatenation can be used if the DataFrames are similar, meaning that they either have the same rows or columns. We will see examples of this later.

To concatenate two DataFrames you will use the function ```pd.concat()```, specifying as arguments the DataFrames to concatenate and ```axis=0``` or ```axis=1``` for vertical or horizontal concatenation, respectively.

Let us first obtain two small DataFrames from the larger `surveys.csv` dataset.

In [ ]:
# Subsetting DataFrames
surveys_df_sub_first10 = surveys_df.head(10)
surveys_df_sub_last10  = surveys_df.tail(10)

We now have two DataFrames, one with the first ten rows of the original dataset, and another with the last ten rows.

### Vertical concatenation

Let's start with **vertical stacking**. In this case the two DataFrames are simply stacked 'on top of' eachother (remember to specify ```axis=0```).
<div>
<img src="images/vertical_stacking.jpeg" width="300"/>
</div>
Vertical stacking can be understood as combining two DataFrames that have different sets of the same type of data. In our example, it may be that one field researcher has registered the first ten entries, and another did the last ten, both using the same laboratory sheets. They both wrote down the same information (weight, species, and so on) of all different individuals. If we combine them, we have one list of twenty records, rather than two lists of ten. 

In [ ]:
# Stack the DataFrames on top of each other
vertical_stack = pd.concat([surveys_df_sub_first10, surveys_df_sub_last10], axis=0)

In [ ]:
print(vertical_stack.info())
vertical_stack

The resulting DataFrame (```vertical_stack```) consists, as expected, of 20 rows. These are the result of the first and last 10 rows of our original DataFrame ```surveys_df```.

You may have noticed that the last ten rows have very high index, not consecutive with the first ten rows. This is because concatenation preserves the indices of the two original DataFrames. If you want a brand new set of indices for your concateneted DataFrame, simply reset the indices using the method ```.reset_index()```. Notice that this adds a column `index` to your DataFrame, that maintains the original index. If you pass `drop=True` into the function, you will avoid the addition of this column.

In [ ]:
vertical_stack.reset_index()

<div class="alert alert-block alert-success">
<b>Exercise 11</b>
    
Now go to the Jupyter Dashboard in your internet browser and continue with exercise 11.


### Horizontal concatenation

It's now time to try **horizontal stacking**. In this case the two DataFrames are simply put one after the other (remember to specify ```axis=1```).
<div>
<img src="images/horizontal_stacking.jpeg" width="300"/>
</div>
Horizontal stacking can be understood as combining two DataFrames that have different measurements on the same observed objects. In our example, it may be that one field researcher has registered the weight and hindfoot length of an individual, and another wrote down their species and sex. They both registered different information of the same individuals. If we combine them, we have one list with all the information of the individual, rather than two lists with partial information.

We now go back to our DataFrames with 10 survey result each, and concatenate those. In this case, as a result, we would expect a DataFrame with the same number of rows of the original ones (10 row) and twice the number of columns (18 columns).

In [ ]:
# Place the DataFrames side by side
horizontal_stack = pd.concat([surveys_df_sub_first10, surveys_df_sub_last10], axis=1)

In [ ]:
print(horizontal_stack.info())
horizontal_stack

Looking at the result of our horizontal concatenation, we may realise that something went wrong. The total number of rows on the resulting DataFrame is 20, instead of 10.

This happens because horizontal stacking will only merge rows that actually "belong together". Rows that relate to the same observed object are merged. To determine this, it compares the indices of the rows. In our two DataFrames, the rows have different indices (1-9 and 35539-35548 respectively). It will therefore not merge any of the rows together, as it does not find any two rows that relate to the same observation.

If we want to force the DataFrames into the form we had in mind, we need to reset the indices of the second DataFrame so that they will match the ones of the first DataFrame.

In [ ]:
surveys_df_sub_last10 = surveys_df_sub_last10.reset_index(drop=True)

In [ ]:
surveys_df_sub_last10

Now that the index has been reset, we can concatenate the two DataFrames.

In [ ]:
horizontal_stack = pd.concat([surveys_df_sub_first10, surveys_df_sub_last10], axis=1)

In [ ]:
print(horizontal_stack.info())
horizontal_stack

<div class="alert alert-block alert-success">
<b>Exercise 12</b>
    
Now go to the Jupyter Dashboard in your internet browser and continue with exercise 12.


## Joining DataFrames

Concatenating DataFrames allows you to combine two entire DataFrames into a single one. In many cases, you want to combine only selected parts of two DataFrames.

You might, for example, want to merge rows of two DataFrames that have matching values in specific columns. The pandas function ```merge()``` performs an operation that you may know as a *join* if you worked with databases before. The join operation joins the content of two DataFrames in a particular way. There are different types of joins, but the workflow to perform a join operation is always the same:

  1. You identify a *left* and a *right* DataFrame, among the two you want to join;
  2. You identify in both your left and right DataFrame a column (or set of columns) to join on;
  3. You choose the type of join;
  4. You perform the join running the function `pd.merge()` with the specified inputs and options.

What it means for a DataFrame to be 'left' or 'right' depends on the type of join, and will become clear in the examples below. For now, just remember that it matters which DataFrame you mention first when performing a join.

Let's see some join example considering two tiny (few rows) DataFrames. Our left DataFrame contains general data of European capitals, and our right DataFrame contains weather measurements for some Dutch towns. We first need to import these datasets:

In [ ]:
left_df = pd.read_csv("../course_materials/data/EU_capitals_tiny.csv", sep=",", header=0)
right_df = pd.read_csv("../course_materials/data/Netherlands_town_weather_tiny.csv", sep=",", header=0)

In [ ]:
left_df

In [ ]:
right_df

The column we want to perform the join on is the one containing information about the town. In the left DataFrame this has name *Capital* while in the right one *Town*.

In [ ]:
inner_join = pd.merge(left_df,right_df,left_on='Capital',right_on='Town',how='inner')
inner_join

As you may notice, the resulting DataFrame has only one line, the only row that the columns *Capital* and *Town* have in common (*Amsterdam*). This is because an inner join selects only those row values that are **the same** in the two columns (mathematically, an intersection).

The columns of the two DataFrames are all preserved, even if they have the same name. In our case, both left and right DataFrames have a column with the same name (*Elevation*). After merging, the two columns are preserved, but with a suffix to distinguish them. If you are not happy with the default suffix, you may specify yours in the list of arguments of the ```pd.merge``` functions.

Let's now look at the other joins:

In [ ]:
left_join = pd.merge(left_df,right_df,left_on='Capital',right_on='Town',how='left')
left_join

In [ ]:
right_join = pd.merge(left_df,right_df,left_on='Capital',right_on='Town',how='right')
right_join

In [ ]:
outer_join = pd.merge(left_df,right_df,left_on='Capital',right_on='Town',how='outer')
outer_join

To resume: a join always merges rows that have matching values in the columns that you merge on. Which rows you find in the resulting DataFrame, depends on the type of join:
- An inner join selects only the rows that result from the combination of matching rows in both the original left and right DataFrames (intersection);
- A left join selects all rows that were in the original left DataFrame, some of which may have been joined with a matching entry from the right DataFrame;
- A right join selects all rows that were in the original right DataFrame, some of which may have been joined with a matching entry from the left DataFrame;
- An outer join merges the two DataFrames and keeps all resulting rows.

To better understand how join works, it may be useful to look at the diagrams below:

<div>
<img src="images/joins.jpeg" width="600"/>
</div>

  - Do you want to select only **common** information between the two DataFrames? Then you use an **inner join**;
  - Do you want to **add information** to your **left** DataFrame? Then you use a **left join**;
  - Do you want to **add information** to your **right** DataFrame? Then you use a **right join**;
  - Do you want to get **all the information** from the two DataFrames? Then you use an **outer join**.

We will continue with [Data visualization](data-science-with-pandas-4.ipynb).